In [1]:
import os
import numpy as np
import math
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras import models, layers, optimizers
import tensorflow as tf
from datetime import datetime

In [2]:
# Importing Model
from Models.models import densenet
model = densenet(121)
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])

In [3]:
#Loading Validation Set

from keras.preprocessing.image import ImageDataGenerator
import os

valid_datagen =  ImageDataGenerator(
    rescale = 1/255.
)

valdir = 'Folds/3folds/fold1/Validation'
valbatchsize = len(os.listdir(valdir+'/True')) + len(os.listdir(valdir+'/False'))

valid_generator = valid_datagen.flow_from_directory(
    valdir,
    batch_size = valbatchsize,
    class_mode = 'binary',
    shuffle = True,
    seed = 123

)

# valid_generator.class_indices

Found 679 images belonging to 3 classes.


In [4]:
lowest_val_loss = float('inf')

In [5]:
trainfalsedirpath = 'Folds/3folds/fold1/Train/False'
trainfalsedir = os.listdir(trainfalsedirpath)

In [6]:
xtrain = np.zeros([2*len(trainfalsedir),600,600,3])
ytrain = np.zeros(2*len(trainfalsedir))

In [7]:
# Loading False samples
for i in range(len(trainfalsedir)):
    xtrain[i] = img_to_array(load_img(trainfalsedirpath+'/'+trainfalsedir[i]))
    ytrain[i] = 0.

In [8]:
traintruedirpath = 'Folds/3folds/fold1/Train/True'
traintruedir = os.listdir(traintruedirpath)

In [9]:
# Loading True samples
start = len(trainfalsedir)
for i in range(len(trainfalsedir), len(trainfalsedir)+len(traintruedir)):
    xtrain[i] = img_to_array(load_img(traintruedirpath+'/'+traintruedir[i-start]))
    ytrain[i] = 1.
    
xtrain = xtrain/255.

In [11]:
training_epochs = 2

In [ ]:
for i in range(training_epochs):
    
    #loading upsampled True samples
    start = len(traintruedir)+len(trainfalsedir)
    for i in range(len(trainfalsedir)+len(traintruedir),xtrain.shape[0]):
        xtrain[i] = img_to_array(load_img(traintruedirpath+'/'+traintruedir[np.random.randint(len(traintruedir))]))/255.
        ytrain[i] = 1.
     
        
    model.fit(
        x = xtrain,
        y = ytrain,
        epochs = 1,
        batch_size = 34,
        shuffle = True,
        verbose = 0, 
    )
    
    val_loss, val_acc = model.evaluate(valid_generator, steps=1)
    print('val_loss:{}\tval_acc:{}'.format(val_loss, val_acc))
    
    f = 1
    if val_loss < lowest_val_loss:
        lowest_val_loss = val_loss
        cpsavepath = 'upsampleddn121checkpoints/'+'fold_{}____'.format(f)+datetime.now().strftime("%d_%m_%Y____%H_%M_%S")
        with open(csavepath+'.json', "w") as json_file:
            json_file.write(model.to_json())
        model.save_weights(csavepath+'.h5')

In [ ]:
truedirpath = 'Folds/3folds/fold1/Train/True'
truedir = os.listdir(truedirpath)
truedir = np.array([int(i[:-4]) for i in truedir])

In [ ]:
falsedirpath = 'Folds/3folds/fold1/Train/False'
falsedir = os.listdir(falsedirpath)
falsedir = np.array([int(i[:-4]) for i in falsedir])

In [ ]:
totalcount = truedir.shape[0]+falsedir.shape[0]

In [ ]:
bs = 32

In [ ]:
numberofiterations = math.ceil((truedir.shape[0] + falsedir.shape[0])/bs)

In [ ]:
from Models.models import dn121scratch as dns

In [ ]:
model = dns()
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
# my_callbacks = [
#     tf.keras.callbacks.ModelCheckpoint(
#         filepath='checkpoints2/'+'fold_{}____'.format(f)+datetime.now().strftime("%d_%m_%Y____%H_%M_%S")+'.h5',
#         monitor = 'val_loss',
#         save_best_only = True
#      ),
# ]

In [ ]:
imagesleft = totalcount
for i in range(numberofiterations):
    
    jcount = min(imagesleft, bs)
    true_batchcount = round(truedir.shape[0]/(truedir.shape[0]+falsedir.shape[0])*jcount)
    false_batchcount = jcount - true_batchcount
    truebatch = np.zeros([true_batchcount,600,600,3])
    falsebatch = np.zeros([false_batchcount,600,600,3])
    trueims = truedir[i*true_batchcount:(i+1)*true_batchcount]
    falseims = falsedir[i*false_batchcount:(i+1)*false_batchcount]
 
#     print(trueims.shape[0], falseims.shape[0])
   
    for j in range(trueims.shape[0]):
        truebatch[j] = img_to_array(load_img(truedirpath+'/'+str(trueims[j])+'.tif'))
    
    for j in range(falseims.shape[0]):
        falsebatch[j] = img_to_array(load_img(falsedirpath+'/'+str(falseims[j])+'.tif'))
        
    batch = np.concatenate([truebatch, falsebatch], axis = 0)
    y = np.concatenate([np.zeros([truebatch.shape[0]]), np.ones([falsebatch.shape[0]])], axis = 0)
    
    permindices = np.random.permutation(batch.shape[0])
    batch = batch[permindices]
    y = y[permindices].reshape(-1,1)
    print(y.shape)
    
    del trueims;del falseims;del truebatch;del falsebatch
    imagesleft-=bs
    
    model.fit(
        x = batch,
        y = y,
        epochs = 1,
        verbose = 2,
    )
    
    del batch;del y;
    

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras.preprocessing.image import array_to_img
plt.imshow(array_to_img(batch[5]))

In [ ]:
a = np.array([7,14,21,28,35,49,56,63,70])
b = np.random.permutation(9)
a = a[b]

print(b)
print(a)